In [1]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import numpy as np

In [2]:
model = tf.keras.models.load_model('combine_prediction_model.keras')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                704       
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 64)                2112      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 128)               8320      
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 1

In [3]:
player_values = input("Please input combine stats in this format\n(POS, Height (in), Weight (lbs), 40 Yard, Bench Press, Vertical Leap (in), Broad Jump (in), Shuttle, 3 Cone)\nEnsure no spaces in between values only seperating values with a comma:\n")
player_values_list = player_values.split(",")

player_POS, player_Height, player_Weight, player_40Yard, player_BenchPress, player_Vert, player_Broad, player_Shuttle, player_3Cone = player_values_list

In [4]:
print(player_POS)

WR


In [5]:
#player_POS = 'CB'
#player_Height = 74.75
#player_Weight = 190
#player_40Yard = 4.41
#player_BenchPress = 21
#player_Vert = 32.5
#player_Broad = 118
#player_Shuttle = 4.51
#player_3Cone = 7.31

In [6]:
holder_df = pd.DataFrame(columns=['POS', "Height (in)", "Weight (lbs)", "40 Yard", "Bench Press", "Vert Leap (in)", "Broad Jump (in)", "Shuttle", "3Cone"])
new_entry = pd.DataFrame({'POS': [player_POS], "Height (in)": [float(player_Height)], "Weight (lbs)": [float(player_Weight)], "40 Yard": [float(player_40Yard)], "Bench Press": [float(player_BenchPress)], "Vert Leap (in)": [float(player_Vert)], "Broad Jump (in)": [float(player_Broad)], "Shuttle": [float(player_Shuttle)], "3Cone": [float(player_3Cone)]})
holder_df = pd.concat([holder_df,new_entry], ignore_index=True)

In [7]:
sampled_df = pd.read_csv('sampled_data.csv')
sampled_df = sampled_df.drop(columns=["Pro Bowl"])
pred_df = pd.concat([sampled_df, holder_df], ignore_index=True)

In [8]:
print(pred_df.head())

  POS  Height (in)  Weight (lbs)  40 Yard  Bench Press  Vert Leap (in)  \
0  OT        78.00         321.0     5.41         20.0            29.0   
1  LB        73.00         235.0     4.70         24.0            33.5   
2  CB        71.00         193.0     4.56          9.0            32.0   
3  OT        78.50         290.0     5.20         22.0            25.0   
4  RB        71.13         208.0     4.39         15.0            37.0   

   Broad Jump (in)  Shuttle  3Cone  
0            102.0     4.90   8.23  
1            114.0     4.33   7.31  
2            114.0     4.29   7.31  
3             99.0     4.83   7.31  
4            121.0     4.39   7.18  


In [9]:
categorical_col = ['POS']
encoder = OneHotEncoder()
encoded_features = encoder.fit_transform(pred_df[categorical_col]).toarray()

In [10]:
numerical_col = ["Height (in)", "Weight (lbs)", "40 Yard", "Bench Press", "Vert Leap (in)", "Broad Jump (in)", "Shuttle", "3Cone"]
scalar = StandardScaler()
scaled_numerical_features = scalar.fit_transform(pred_df[numerical_col])

In [11]:
pred_data = pd.concat([pd.DataFrame(encoded_features), pd.DataFrame(scaled_numerical_features)], axis=1)
print(pred_data.head())

    0    1    2    3    4    5    6    7    8    9   ...   11   12        0   \
0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  ...  0.0  0.0  1.591878   
1  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  ...  0.0  0.0 -0.308349   
2  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0 -1.068439   
3  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  ...  0.0  0.0  1.781900   
4  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  ...  0.0  0.0 -1.019033   

         1         2         3         4         5         6         7   
0  1.819784  2.224586 -0.011419 -0.923683 -1.440537  2.249537  3.096946  
1 -0.119642 -0.295699  0.695189  0.206501 -0.035737 -0.287157  0.086990  
2 -1.066803 -0.792656 -1.954589 -0.170227 -0.035737 -0.465171  0.086990  
3  1.120689  1.479149  0.341885 -1.928291 -1.791737  1.938014  0.086990  
4 -0.728531 -1.396105 -0.894678  1.085534  0.783730 -0.020137 -0.338329  

[5 rows x 21 columns]


In [12]:
pred_input = pred_data.tail(1)
print(pred_input.to_string())

       0    1    2    3    4    5    6    7    8    9    10   11   12        0        1         2         3        4         5         6        7 
1840  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0 -0.213337 -0.86384 -1.254117 -0.011419  1.21111  1.369063  0.291387  0.08699


In [13]:
X_predict = np.array(pred_input)

In [14]:
prediction = model.predict(X_predict)
print("prediction:", prediction)

1/1 [==============================] - 0s 44ms/step
prediction: [[0.55585146]]


In [15]:
#2020,WR,73.5,208,4.59,20,31.5,117,4.42,7.31
#2020,WR,73.25,202,4.43,20,37.5,126,4.46,7.31
#2022,RB,72.38,211,4.48,21,30.0,117,4.51,7.31

In [16]:
#2021,WR,71.5,197,4.61,20,38.5,127,4.26,6.9,1
#2021,WR,70.13,180,4.48,20,33.0,122,4.21,6.86,0
#2021,WR,75.0,211,4.49,12,37.0,124,4.43,7.2,0
#2021,WR,72.75,203,4.47,17,40.0,124,4.14,7.02,0